In [2]:
!pip install ludwig

In [3]:
# from ludwig.api import LudwigModel
# import pandas

# df = pandas.read_csv('')
# df.head()


from ludwig.datasets import agnews

# Loads the dataset as a pandas.DataFrame
train_df, test_df, _ = agnews.load(split=True)

# Prints a preview of the first five rows.
train_df.head(5)

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(
train.csv: 28.1MB [00:00, 85.1MB/s]                            
test.csv: 1.77MB [00:00, 8.38MB/s]                            


,class_index,title,description,class
4059,4,News: DIY phishing kits hit the Net,Do-it-yourself phishing kits are being made av...,sci_tech
4080,4,Two NCSSM students win national team science c...,Two students at the Durham NC School of Scienc...,sci_tech
4108,4,Web Sites Agree to Be Accessible to Blind (AP),AP - In one of the first enforcement actions o...,sci_tech
4109,4,European Winters Could Disappear by 2080 - Rep...,Reuters - Europe is warming up\more quickly th...,sci_tech
4110,4,Charter Schools and Testing Collide,The education policy of the Bush administratio...,sci_tech


In [4]:
config = {
  "input_features": [
    {
      "name": "title",            # The name of the input column
      "type": "text",             # Data type of the input column
      "encoder": {
        "type": "parallel_cnn",  # The model architecture we should use for
      }                          # encoding this column
    }
  ],
  "output_features": [
    {
      "name": "class",
      "type": "category",
    }
  ],
  "trainer": {
    "epochs": 3,  # We'll train for three epochs. Training longer might give
                  # better performance.
  }
}

In [5]:
import logging
from ludwig.api import LudwigModel

# Constructs Ludwig model from config dictionary
model = LudwigModel(config, logging_level=logging.INFO)

2024-03-23 23:43:38,914	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-03-23 23:43:39,331	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [6]:
train_stats, preprocessed_data, output_directory = model.train(dataset=train_df)

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/Convolution.cpp:1008.)
  return F.conv1d(input, weight, bias, self.stride,
2024-03-23 23:44:37.406293: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 23:44:37.406445: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 23:44:37.546034: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Training: 100%|██████████| 30/30 [00:22<00:00,  1.36it/s, loss=0.416]


In [7]:
import pandas as pd

text_to_predict = pd.DataFrame({
  "title": [
    "Google may spur cloud cybersecurity M&A with $5.4B Mandiant buy",
    "Europe struggles to meet mounting needs of Ukraine's fleeing millions",
    "How the pandemic housing market spurred buyer's remorse across America",
  ]
})

predictions, output_directory = model.predict(text_to_predict)

predictions

Prediction: 100%|██████████| 1/1 [00:00<00:00, 83.21it/s]


,class_predictions,class_probabilities,class_probability,class_probabilities_sci_tech,class_probabilities_world,class_probabilities_sports,class_probabilities_business
0,business,"[0.3130781948566437, 0.00023208462516777217, 3...",0.686654,0.313078,0.000232,0.000036,0.686654
1,world,"[0.0031794668175280094, 0.9947161078453064, 0....",0.994716,0.003179,0.994716,0.000799,0.001306
2,sci_tech,"[0.563317596912384, 0.06807877868413925, 5.902...",0.563318,0.563318,0.068079,0.000059,0.368545
